In [16]:
import string

class Vectorizer:
  def standardize(self, text):
    text = text.lower()
    return "".join(char for char in text if char not in string.punctuation)

  def tokenize(self, text):
    return text.split()

  import string

class Vectorizer:
  def standardize(self, text):
    text = text.lower()
    return "".join(char for char in text if char not in string.punctuation)

  def tokenize(self, text):
    return text.split()

  def make_vocabulary(self, dataset):
    self.vocabulary = {"": 0, "[UNK]" : 1}
    for text in dataset:
      text = self.standardize(text)
      tokens = self.tokenize(text)
      for token in tokens:
        if token not in self.vocabulary:
          self.vocabulary[token] = len(self.vocabulary)
    self.inverse_vocabulary = dict(
        (v, k) for k, v in self.vocabulary.items()
    )

  def encode(self, text):
    text = self.standardize(text)
    tokens = self.tokenize(text)
    return [self.vocabulary.get(token, 1) for token in tokens]

  def decode(self, int_sequence):
    return " ".join(
        self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence
    )


vectorizer = Vectorizer()

dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy bloons."
]

vectorizer.make_vocabulary(dataset)

In [17]:
test_sentence = "I write, rewrite, and still rewrite agiain"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

[2, 3, 5, 7, 1, 5, 1]
i write rewrite and [UNK] rewrite [UNK]


In [18]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    output_mode = "int"
)

In [19]:
import re
import tensorflow as tf

def custom_standardization_fn(string_tensor):
  lowercase_string = tf.strings.lower(string_tensor)
  return tf.strings.regex_replace(
      lowercase_string, f"[{re.escape(string.punctuation)}]", ""
  )

def custom_split_fn(string_tensor):
  return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode = "int",
    standardize = custom_standardization_fn,
    split = custom_split_fn
)

In [20]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms"
]
text_vectorization.adapt(dataset)

In [21]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [22]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)
i write rewrite and [UNK] rewrite again


In [23]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  8646k      0  0:00:09  0:00:09 --:--:-- 16.8M


In [24]:
!rm -r aclImdb/train/unsup

In [25]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [28]:
# 훈련 세트에서 일부 데이터를 추출하여 새로운 검증 세트를 생성
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb") # "aclImdb"라는 이름의 디렉토리에 대한 경로 객체를 생성
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category) # 특정 디렉토리 내의 모든 파일과 디렉토리의 이름을 리스트로 가져옵니다.
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:  # shutil 모듈의 move 함수를 사용하여 파일을 하나의 위치에서 다른 위치로 이동
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

FileExistsError: ignored

In [29]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [30]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size = batch_size)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size = batch_size
)

Found 25000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [31]:
for inputs, targets in train_ds:
  print("inputs.shape : ", inputs.shape)
  print("inputs.dtype : ", inputs.dtype)
  print("targets.shape: ", targets.shape)
  print("targets.dtype : ", targets.dtype)
  print("inputs[0] : ", inputs[0])
  print("targets[0] : ", targets[0])
  break


inputs.shape :  (32,)
inputs.dtype :  <dtype: 'string'>
targets.shape:  (32,)
targets.dtype :  <dtype: 'int32'>
inputs[0] :  tf.Tensor(b"This is the worst sequel on the face of the world of movies. Once again it doesn't make since. The killer still kills for fun. But this time he is killing people that are making a movie about what happened in the first movie. Which means that it is the stupidest movie ever.<br /><br />Don't watch this. If you value the one precious hour during this movie then don't watch it. You'll want to ask the director and the person beside you what made him make it. Because it just doesn't combine the original makes of horror, action, and crime.<br /><br />Don't let your children watch this. Teenager, young child or young adult, this movie has that sorta impact upon people.", shape=(), dtype=string)
targets[0] :  tf.Tensor(0, shape=(), dtype=int32)


**단어를 집합으로 처리하기: BoW 방식**

Single words (unigrams) with binary encoding


TextVectorization 층으로 데이터 전처리하기

In [32]:
# TextVectorization 레이어를 사용하여 텍스트 데이터를 전처리
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot", # Multi-hot 인코딩은 텍스트에서 각 단어가 존재하는지 여부만 표시, 각 단어가 문서에 포함되면 1, 그렇지 않으면 0
)
text_only_train_ds = train_ds.map(lambda x, y: x) # 텍스트 데이터만 사용하여 어휘를 구축
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), # 입력 텍스트 x를 multi-hot 인코딩으로 변환하고 레이블 y를 그대로 둔다.
    num_parallel_calls=4) # num_parallel_calls=4는 동시에 4개의 병렬 작업을 사용하여 map 함수를 실행
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

모델 생성 유틸리티

In [33]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

이진 유니그램 모델 훈련하고 테스트하기

In [34]:
model = get_model() # get_model()을 호출하여 딥 러닝 모델을 생성
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True) # 모델의 검증 성능이 향상될 때만 가중치를 저장
]
model.fit(binary_1gram_train_ds.cache(), # .cache() 메서드는 데이터를 메모리에 캐시하여 데이터 로딩 속도를 높이는 역할
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")
print(f"테스트 정확도: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
782/782 [==============================] - 18s 21ms/step - loss: 0.3964 - accuracy: 0.8350 - val_loss: 0.2318 - val_accuracy: 0.9254
Epoch 2/10
782/782 [=======================

이진 인코딩을 사용한 바이그램

**바이그램을 반환하는 TextVectorization 층 만들기**

In [35]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

**이진 바이그램 모델 훈련하고 테스트하기**

In [36]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras",
                                    save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"테스트 정확도: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3673 - accuracy: 0.8502 - val_loss: 0.1810 - val_accuracy: 0.9422
Epoch 2/10
782/782 [=====================

**TF-IDF 인코딩을 사용한 바이그램**

토큰 카운트를 반환하는 TextVectorization 층

In [37]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="count"
)

TF-IDF 가중치가 적용된 출력을 반환하는 TextVectorization 층

In [38]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)


TF-IDF 바이그램 모델 훈련하고 테스트하기

In [39]:
# 텐서플로 2.8.x 버전에서 TF-IDF 인코딩을 GPU에서 수행할 때 오류가 발생할 수 있습니다.
# 텐서플로 2.9에서 이 이슈가 해결되었지만 코드를 테스트할 시점에 코랩의 텐서플로 버전은 2.8.2이기 때문에
# 에러를 피하기 위해 CPU를 사용하여 텍스트를 변환합니다.
import tensorflow as tf

with tf.device("cpu"):
    text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"테스트 정확도: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
782/782 [==============================] - 13s 15ms/step - loss: 0.4809 - accuracy: 0.7978 - val_loss: 0.2455 - val_accuracy: 0.9092
Epoch 2/10
782/782 [=====================

In [40]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [41]:
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"긍정적인 리뷰일 확률: {float(predictions[0] * 100):.2f} 퍼센트")

긍정적인 리뷰일 확률: 99.29 퍼센트
